In [1]:
import pyspark

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('Regression Algos').getOrCreate()

24/04/18 20:48:32 WARN Utils: Your hostname, PF0P0YDX resolves to a loopback address: 127.0.1.1; using 192.168.1.23 instead (on interface wlp5s0)
24/04/18 20:48:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/18 20:48:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/04/18 20:48:39 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
df = spark.read.csv('file:///home/user1/Desktop/Algos Hands-on/tips.csv',header='True',inferSchema='True')

In [5]:
df2 = spark.read.csv('file:///home/user1/Desktop/Algos Hands-on/heart.csv',header='True',inferSchema='True')

In [7]:
### Linear Regression

df.show()

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|
|     18.43| 3.0|  Male|    No|Sun|Dinner|   4|
|     14.83|3.02|Female|    No|Sun|Dinner|   2|
|     21.58|3.92|  Male|    No|Sun|Dinner|   2|
|     10.33|1.67|Female|    No|Sun|Dinner|   3|
|     16.29|3.71|  Male|    No|Sun|Dinne

In [8]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCols=['sex','smoker','day','time'],outputCols=['sex_numeric','smoker_numeric','day_numeric','time_numeric'])
indexed_df = indexer.fit(df).transform(df)
indexed_df.show()

+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_numeric|smoker_numeric|day_numeric|time_numeric|
+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|           0.0|        1.0|         0.0|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|         0.0|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|           0.0|        1.0|         0.0|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|        0.0|           0.0|        1.0|         0.0|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|         0.0|


In [9]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=['tip','size','sex_numeric','smoker_numeric','day_numeric','time_numeric'],outputCol='features')
vector_df = assembler.transform(indexed_df)
vector_df.show()

+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+--------------------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_numeric|smoker_numeric|day_numeric|time_numeric|            features|
+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+--------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|           0.0|        1.0|         0.0|[1.01,2.0,1.0,0.0...|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|[1.66,3.0,0.0,0.0...|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|[3.5,3.0,0.0,0.0,...|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|         0.0|[3.31,2.0,0.0,0.0...|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|           0.0|        1.0|         0.0|[3.61,4.0,1.0,0.0...|
|     25.29|4.71|  Male|    No|S

In [10]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

train_data , test_data = vector_df.randomSplit([0.7,0.3])
lg = LinearRegression(featuresCol='features',labelCol='total_bill')
regressor = lg.fit(train_data)
predictions = regressor.transform(test_data)
evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="total_bill", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print(rmse)

24/04/18 20:49:36 WARN Instrumentation: [790ac6cc] regParam is zero, which might cause numerical instability and overfitting.
24/04/18 20:49:38 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/04/18 20:49:38 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


5.688636441818585


In [11]:
predictions.select('total_bill','prediction').show()

+----------+------------------+
|total_bill|        prediction|
+----------+------------------+
|      7.51| 13.73645822998812|
|      8.52|12.277642321947535|
|      8.58|11.485628163078381|
|      9.55|14.245951102640147|
|       9.6|  21.3870798660935|
|      9.94| 14.19846863285899|
|     10.27|14.619280914024543|
|     10.33|16.520781139640786|
|     10.34| 17.85538071038733|
|     10.34| 15.44251007908721|
|     10.63|16.132327593188684|
|     11.17| 10.97109690327869|
|     11.24| 16.82168166614805|
|     11.59|16.092273712127756|
|     11.61|19.688456605714634|
|     11.87|11.335800880288836|
|     12.26|12.373804507163868|
|     12.46|13.317987435120976|
|     12.54|16.835558928163124|
|     12.66|16.835558928163124|
+----------+------------------+
only showing top 20 rows



In [12]:
predictions.count()

74

In [13]:
### Logistic Regression

df2.show()

+---+------+-------------+---------+-----------+---------+----------+-----+--------------+-------+--------+------------+
|Age|Gender|ChestPainType|RestingBP|Cholesterol|FastingBS|RestingECG|MaxHR|ExerciseAngina|Oldpeak|ST_Slope|HeartDisease|
+---+------+-------------+---------+-----------+---------+----------+-----+--------------+-------+--------+------------+
| 40|     M|          ATA|      140|        289|        0|    Normal|  172|             N|    0.0|      Up|           0|
| 49|     F|          NAP|      160|        180|        0|    Normal|  156|             N|    1.0|    Flat|           1|
| 37|     M|          ATA|      130|        283|        0|        ST|   98|             N|    0.0|      Up|           0|
| 48|     F|          ASY|      138|        214|        0|    Normal|  108|             Y|    1.5|    Flat|           1|
| 54|     M|          NAP|      150|        195|        0|    Normal|  122|             N|    0.0|      Up|           0|
| 39|     M|          NAP|      

In [14]:
df2.printSchema()

root
 |-- Age: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- ChestPainType: string (nullable = true)
 |-- RestingBP: integer (nullable = true)
 |-- Cholesterol: integer (nullable = true)
 |-- FastingBS: integer (nullable = true)
 |-- RestingECG: string (nullable = true)
 |-- MaxHR: integer (nullable = true)
 |-- ExerciseAngina: string (nullable = true)
 |-- Oldpeak: double (nullable = true)
 |-- ST_Slope: string (nullable = true)
 |-- HeartDisease: integer (nullable = true)



In [15]:
indexer = StringIndexer(inputCols=['Gender','ChestPainType','RestingECG','ExerciseAngina','ST_Slope'],outputCols=['Gender_numeric','CPT','RECG','EA','ST_S'])
indexed_df = indexer.fit(df2).transform(df2)
indexed_df.show()

+---+------+-------------+---------+-----------+---------+----------+-----+--------------+-------+--------+------------+--------------+---+----+---+----+
|Age|Gender|ChestPainType|RestingBP|Cholesterol|FastingBS|RestingECG|MaxHR|ExerciseAngina|Oldpeak|ST_Slope|HeartDisease|Gender_numeric|CPT|RECG| EA|ST_S|
+---+------+-------------+---------+-----------+---------+----------+-----+--------------+-------+--------+------------+--------------+---+----+---+----+
| 40|     M|          ATA|      140|        289|        0|    Normal|  172|             N|    0.0|      Up|           0|           0.0|2.0| 0.0|0.0| 1.0|
| 49|     F|          NAP|      160|        180|        0|    Normal|  156|             N|    1.0|    Flat|           1|           1.0|1.0| 0.0|0.0| 0.0|
| 37|     M|          ATA|      130|        283|        0|        ST|   98|             N|    0.0|      Up|           0|           0.0|2.0| 2.0|0.0| 1.0|
| 48|     F|          ASY|      138|        214|        0|    Normal|  108| 

In [16]:
indexed_df.columns

['Age',
 'Gender',
 'ChestPainType',
 'RestingBP',
 'Cholesterol',
 'FastingBS',
 'RestingECG',
 'MaxHR',
 'ExerciseAngina',
 'Oldpeak',
 'ST_Slope',
 'HeartDisease',
 'Gender_numeric',
 'CPT',
 'RECG',
 'EA',
 'ST_S']

In [17]:
assembler = VectorAssembler(inputCols=['Age','Gender_numeric','CPT','RestingBP','Cholesterol','FastingBS','RECG','MaxHR','EA','Oldpeak','ST_S'],outputCol='features')
vector_df = assembler.transform(indexed_df)
vector_df.show()

+---+------+-------------+---------+-----------+---------+----------+-----+--------------+-------+--------+------------+--------------+---+----+---+----+--------------------+
|Age|Gender|ChestPainType|RestingBP|Cholesterol|FastingBS|RestingECG|MaxHR|ExerciseAngina|Oldpeak|ST_Slope|HeartDisease|Gender_numeric|CPT|RECG| EA|ST_S|            features|
+---+------+-------------+---------+-----------+---------+----------+-----+--------------+-------+--------+------------+--------------+---+----+---+----+--------------------+
| 40|     M|          ATA|      140|        289|        0|    Normal|  172|             N|    0.0|      Up|           0|           0.0|2.0| 0.0|0.0| 1.0|(11,[0,2,3,4,7,10...|
| 49|     F|          NAP|      160|        180|        0|    Normal|  156|             N|    1.0|    Flat|           1|           1.0|1.0| 0.0|0.0| 0.0|[49.0,1.0,1.0,160...|
| 37|     M|          ATA|      130|        283|        0|        ST|   98|             N|    0.0|      Up|           0|     

In [18]:
vector_df.select('features','HeartDisease').show(truncate=False)

+----------------------------------------------------+------------+
|features                                            |HeartDisease|
+----------------------------------------------------+------------+
|(11,[0,2,3,4,7,10],[40.0,2.0,140.0,289.0,172.0,1.0])|0           |
|[49.0,1.0,1.0,160.0,180.0,0.0,0.0,156.0,0.0,1.0,0.0]|1           |
|[37.0,0.0,2.0,130.0,283.0,0.0,2.0,98.0,0.0,0.0,1.0] |0           |
|[48.0,1.0,0.0,138.0,214.0,0.0,0.0,108.0,1.0,1.5,0.0]|1           |
|(11,[0,2,3,4,7,10],[54.0,1.0,150.0,195.0,122.0,1.0])|0           |
|(11,[0,2,3,4,7,10],[39.0,1.0,120.0,339.0,170.0,1.0])|0           |
|[45.0,1.0,2.0,130.0,237.0,0.0,0.0,170.0,0.0,0.0,1.0]|0           |
|(11,[0,2,3,4,7,10],[54.0,2.0,110.0,208.0,142.0,1.0])|0           |
|(11,[0,3,4,7,8,9],[37.0,140.0,207.0,130.0,1.0,1.5]) |1           |
|[48.0,1.0,2.0,120.0,284.0,0.0,0.0,120.0,0.0,0.0,1.0]|0           |
|[37.0,1.0,1.0,130.0,211.0,0.0,0.0,142.0,0.0,0.0,1.0]|0           |
|[58.0,0.0,2.0,136.0,164.0,0.0,2.0,99.0,1.0,2.0,

In [19]:
model_df = vector_df.select('features','HeartDisease')

In [20]:
train_data , test_data = model_df.randomSplit([0.7,0.3])

In [21]:
from pyspark.ml.classification import LogisticRegression

log_reg = LogisticRegression(labelCol='HeartDisease').fit(train_data)

In [22]:
training_results = log_reg.evaluate(train_data).predictions

In [23]:
training_results.count()

668

In [24]:
training_results.show()

+--------------------+------------+--------------------+--------------------+----------+
|            features|HeartDisease|       rawPrediction|         probability|prediction|
+--------------------+------------+--------------------+--------------------+----------+
|(11,[0,1,2,3,4,7]...|           0|[1.71510772068085...|[0.84749760606383...|       0.0|
|(11,[0,1,2,3,4,7]...|           0|[1.94667771970879...|[0.87508392887810...|       0.0|
|(11,[0,1,2,3,4,7]...|           0|[2.07966160337693...|[0.88891062151753...|       0.0|
|(11,[0,1,2,3,4,7]...|           0|[2.74856024807007...|[0.93983198652506...|       0.0|
|(11,[0,1,2,3,5,7]...|           1|[0.65423965253620...|[0.65796522169890...|       0.0|
|(11,[0,1,3,4,7,8]...|           1|[-0.4905227129532...|[0.37977043757614...|       1.0|
|(11,[0,1,3,4,7,9]...|           1|[0.00840509841966...|[0.50210126223450...|       0.0|
|(11,[0,1,3,4,7,9]...|           0|[0.04475534729868...|[0.51118696955701...|       0.0|
|(11,[0,1,3,4,7,9]...

In [25]:
results = log_reg.evaluate(test_data)

In [26]:
results.predictions.show()

+--------------------+------------+--------------------+--------------------+----------+
|            features|HeartDisease|       rawPrediction|         probability|prediction|
+--------------------+------------+--------------------+--------------------+----------+
|(11,[0,1,3,4,7,8]...|           1|[-0.3257883573806...|[0.41926573215692...|       1.0|
|(11,[0,1,3,4,7,9]...|           0|[-1.0117373277600...|[0.26663999059319...|       1.0|
|(11,[0,1,3,4,7,10...|           0|[2.85337410070588...|[0.94549283250333...|       0.0|
|(11,[0,1,3,5,7],[...|           1|[-1.9607157438686...|[0.12338960853573...|       1.0|
|(11,[0,1,3,7,8,9]...|           1|[-2.3761307103545...|[0.08501105235719...|       1.0|
|(11,[0,2,3,4,5,7]...|           1|[-1.4989771145768...|[0.18257813309874...|       1.0|
|(11,[0,2,3,4,5,7]...|           1|[-1.7533380975920...|[0.14762666082861...|       1.0|
|(11,[0,2,3,4,6,7]...|           0|[2.01639410171852...|[0.88250763646614...|       0.0|
|(11,[0,2,3,4,7],[...

In [27]:
predictions.count()

74